In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import sys
import numpy as np
import matplotlib.pyplot as plt

from nets import *
from cfgs import *
from data import *
from clip_ops.clip_ops import *
from trainer import *


In [ ]:
%matplotlib inline
save_plot = False
plt.rcParams.update({'font.size': 10, 'axes.labelsize': 'x-large'})
D = 201

# v1, v2を[0, 1]の範囲で生成
x = np.linspace(0, 1.0, D)

# v3を複数の値でサンプリング（3次元可視化のため）
c = 10.0
v3_values = np.linspace(0, c, 5)  # v3を0, c/4, c/2, 3c/4, cの5つの値でサンプリング
print(f'v3_values: {v3_values}')

cfg = additive_1x3_constrained_c10_config.cfg
cfg.test.num_misreports = 1
cfg.test.gd_iter = 0
cfg.test.batch_size = D
cfg.test.save_output = True


In [ ]:
Net = additive_net.Net
Generator = constrained_3item_generator.Generator
clip_op_lambda = (lambda x: clip_op_constrained_3item(x, c=cfg.c))
Trainer = trainer.Trainer


In [ ]:
# 各v3値に対してテストデータを生成し、推論を実行
# 各v3値での結果を保存
alloc_dict = {}
pay_dict = {}

for v3_val in v3_values:
    # 3財のテストデータを生成: [v1, v2, v3]
    X_tst = np.stack([v.flatten() for v in np.meshgrid(x, x)], axis=-1)
    # v3を固定値で追加
    X_tst = np.concatenate([X_tst, np.full((X_tst.shape[0], 1), v3_val)], axis=-1)
    # regretNet用に形状を変換: [batch_size, num_agents, num_items]
    X_tst = np.expand_dims(X_tst, 1)  # [batch_size, 1, 3]
    
    # 推論を実行（各v3値ごとに新しいnetとtrainerを作成）
    net = Net(cfg)
    m = Trainer(cfg, "test", net, clip_op_lambda)
    generator = Generator(cfg, 'test', X_tst)
    cfg.test.num_batches = int(X_tst.shape[0]/cfg.test.batch_size)
    m.test(generator)
    
    # 結果を即座に読み込み（次のテストで上書きされる前に保存）
    alloc_file = os.path.join(cfg.dir_name, "alloc_tst_" + str(cfg.test.restore_iter) + ".npy")
    pay_file = os.path.join(cfg.dir_name, "pay_tst_" + str(cfg.test.restore_iter) + ".npy")
    
    # ファイルが存在することを確認
    if os.path.exists(alloc_file) and os.path.exists(pay_file):
        alloc = np.load(alloc_file)  # [D*D, num_agents, num_items] -> [D*D, 1, 3]
        pay = np.load(pay_file)  # [D*D, num_agents] -> [D*D, 1]
        
        # 単一bidderの場合、形状を変換
        alloc = alloc[:, 0, :].reshape(D, D, 3)  # [D, D, 3]
        pay = pay[:, 0].reshape(D, D)  # [D, D]
        
        alloc_dict[v3_val] = alloc.copy()  # コピーを作成
        pay_dict[v3_val] = pay.copy()  # コピーを作成
        print(f"Completed v3 = {v3_val:.2f}")
    else:
        print(f"Error: Files not found for v3 = {v3_val:.2f}")


In [ ]:
# データは既に読み込まれているので、ここでは何もしない
# alloc_dictとpay_dictに各v3値での結果が保存されている


In [ ]:
# 財1の配分確率を複数のv3値で可視化
plt.rcParams.update({'font.size': 10, 'axes.labelsize': 'x-large'})
fig, axes = plt.subplots(ncols=len(v3_values), nrows=1, figsize=(4*len(v3_values), 4))

for idx, v3_val in enumerate(v3_values):
    alloc = alloc_dict[v3_val]
    ax = axes[idx]
    img = ax.imshow(alloc[::-1, :, 0], extent=[0, 1, 0, 1], vmin=0.0, vmax=1.0, cmap='YlOrRd')
    ax.set_xlabel(r'$v_1$')
    if idx == 0:
        ax.set_ylabel(r'$v_2$')
    ax.set_title(f'v3={v3_val:.2f}')
    plt.colorbar(img, ax=ax, fraction=0.046, pad=0.04)

plt.suptitle(f'Prob. of allocating item 1 (c={c})', y=1.02)
plt.tight_layout()

if save_plot:
    plt.savefig(os.path.join(cfg.dir_name, 'alloc1_3d.pdf'), bbox_inches='tight', pad_inches=0.05)


In [ ]:
# 財2の配分確率を複数のv3値で可視化
plt.rcParams.update({'font.size': 10, 'axes.labelsize': 'x-large'})
fig, axes = plt.subplots(ncols=len(v3_values), nrows=1, figsize=(4*len(v3_values), 4))

for idx, v3_val in enumerate(v3_values):
    alloc = alloc_dict[v3_val]
    ax = axes[idx]
    img = ax.imshow(alloc[::-1, :, 1], extent=[0, 1, 0, 1], vmin=0.0, vmax=1.0, cmap='YlOrRd')
    ax.set_xlabel(r'$v_1$')
    if idx == 0:
        ax.set_ylabel(r'$v_2$')
    ax.set_title(f'v3={v3_val:.2f}')
    plt.colorbar(img, ax=ax, fraction=0.046, pad=0.04)

plt.suptitle(f'Prob. of allocating item 2 (c={c})', y=1.02)
plt.tight_layout()

if save_plot:
    plt.savefig(os.path.join(cfg.dir_name, 'alloc2_3d.pdf'), bbox_inches='tight', pad_inches=0.05)


In [ ]:
# 財3の配分確率を複数のv3値で可視化
plt.rcParams.update({'font.size': 10, 'axes.labelsize': 'x-large'})
fig, axes = plt.subplots(ncols=len(v3_values), nrows=1, figsize=(4*len(v3_values), 4))

for idx, v3_val in enumerate(v3_values):
    alloc = alloc_dict[v3_val]
    ax = axes[idx]
    img = ax.imshow(alloc[::-1, :, 2], extent=[0, 1, 0, 1], vmin=0.0, vmax=1.0, cmap='YlOrRd')
    ax.set_xlabel(r'$v_1$')
    if idx == 0:
        ax.set_ylabel(r'$v_2$')
    ax.set_title(f'v3={v3_val:.2f}')
    plt.colorbar(img, ax=ax, fraction=0.046, pad=0.04)

plt.suptitle(f'Prob. of allocating item 3 (c={c})', y=1.02)
plt.tight_layout()

if save_plot:
    plt.savefig(os.path.join(cfg.dir_name, 'alloc3_3d.pdf'), bbox_inches='tight', pad_inches=0.05)


In [ ]:
# Paymentを複数のv3値で可視化
plt.rcParams.update({'font.size': 10, 'axes.labelsize': 'x-large'})

# すべてのv3値でのpaymentの最大値と最小値を計算（色スケール統一のため）
pay_min = min([np.min(pay_dict[v3_val]) for v3_val in v3_values])
pay_max = max([np.max(pay_dict[v3_val]) for v3_val in v3_values])

fig, axes = plt.subplots(ncols=len(v3_values), nrows=1, figsize=(4*len(v3_values), 4))

for idx, v3_val in enumerate(v3_values):
    pay = pay_dict[v3_val]
    ax = axes[idx]
    img = ax.imshow(pay[::-1, :], extent=[0, 1, 0, 1], cmap='YlOrRd', vmin=pay_min, vmax=pay_max)
    ax.set_xlabel(r'$v_1$')
    if idx == 0:
        ax.set_ylabel(r'$v_2$')
    ax.set_title(f'v3={v3_val:.2f}')
    plt.colorbar(img, ax=ax, fraction=0.046, pad=0.04)

plt.suptitle(f'Payment (c={c})', y=1.02)
plt.tight_layout()

if save_plot:
    plt.savefig(os.path.join(cfg.dir_name, 'pay_3d.pdf'), bbox_inches='tight', pad_inches=0.05)


In [ ]:
# 制約違反の確認（各v3値に対して）
print("Constraint Violation Statistics for each v3:")
for v3_val in v3_values:
    alloc = alloc_dict[v3_val]
    # 下界: max(0, alloc1 + alloc2 - 1) <= alloc3
    # 上界: alloc3 <= min(alloc1, alloc2)
    lower_bound = np.maximum(0, alloc[:, :, 0] + alloc[:, :, 1] - 1)
    upper_bound = np.minimum(alloc[:, :, 0], alloc[:, :, 1])
    lower_violation = np.maximum(0, lower_bound - alloc[:, :, 2])
    upper_violation = np.maximum(0, alloc[:, :, 2] - upper_bound)
    constraint_violation = lower_violation + upper_violation
    
    print(f"\nv3 = {v3_val:.2f}:")
    print(f"  Min: {np.min(constraint_violation):.6f}")
    print(f"  Max: {np.max(constraint_violation):.6f}")
    print(f"  Mean: {np.mean(constraint_violation):.6f}")
    print(f"  Number of violations: {np.sum(constraint_violation > 1e-6)}")


In [ ]:
# 制約違反を複数のv3値で可視化
plt.rcParams.update({'font.size': 10, 'axes.labelsize': 'x-large'})
fig, axes = plt.subplots(ncols=len(v3_values), nrows=1, figsize=(4*len(v3_values), 4))

for idx, v3_val in enumerate(v3_values):
    alloc = alloc_dict[v3_val]
    lower_bound = np.maximum(0, alloc[:, :, 0] + alloc[:, :, 1] - 1)
    upper_bound = np.minimum(alloc[:, :, 0], alloc[:, :, 1])
    lower_violation = np.maximum(0, lower_bound - alloc[:, :, 2])
    upper_violation = np.maximum(0, alloc[:, :, 2] - upper_bound)
    constraint_violation = lower_violation + upper_violation
    
    ax = axes[idx]
    img = ax.imshow(constraint_violation[::-1, :], extent=[0, 1, 0, 1], cmap='Reds')
    ax.set_xlabel(r'$v_1$')
    if idx == 0:
        ax.set_ylabel(r'$v_2$')
    ax.set_title(f'v3={v3_val:.2f}')
    plt.colorbar(img, ax=ax, fraction=0.046, pad=0.04)

plt.suptitle(f'Constraint Violation (c={c})', y=1.02)
plt.tight_layout()

if save_plot:
    plt.savefig(os.path.join(cfg.dir_name, 'constraint_violation_3d.pdf'), bbox_inches='tight', pad_inches=0.05)


In [ ]:
# Payment statistics（各v3値に対して）
v_max = x[-1]
print("Payment Statistics for each v3:")
for v3_val in v3_values:
    pay = pay_dict[v3_val]
    print(f"\nv3 = {v3_val:.2f}:")
    print(f"  Min: {np.min(pay):.6f}")
    print(f"  Max: {np.max(pay):.6f}")
    print(f"  Mean: {np.mean(pay):.6f}")
    print(f"  Median: {np.median(pay):.6f}")
    print(f"  Std: {np.std(pay):.6f}")
    print(f"  Payment at selected points:")
    print(f"    pay[0, 0] (v1=0, v2=0): {pay[0, 0]:.6f}")
    print(f"    pay[D//2, D//2] (v1={v_max/2:.1f}, v2={v_max/2:.1f}): {pay[D//2, D//2]:.6f}")
    print(f"    pay[-1, -1] (v1={v_max:.1f}, v2={v_max:.1f}): {pay[-1, -1]:.6f}")
